### Importing the required modules

In [ ]:
import sys
import torch
from pathlib import Path
from typing import Optional
from trl import SFTConfig, SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Add the parent directory to the system path
sys.path.append(str(Path().resolve().parent.parent))

# Import local dependencies
from src.utils import get_device, set_seed
from src.hf import hf_login, load_hf_dataset, dataset_to_pandas

### Setting up the environment

In [ ]:
# Login to Hugging Face
hf_login()

In [ ]:
# Get the device available on the system
device = get_device()
use_cuda = torch.cuda.is_available() and "cuda" in str(device).lower()

# Print the detected device
print(f"Detected device: {device}")

### Constants, hyperparameters and model configurations

In [ ]:
seed = 42 # Seed for reproducibility
test_size = 0.2 # Train-test split percentage
model_id = "HuggingFaceTB/SmolLM2-135M" # The model ID
dataset_name = "banghua/DL-SFT-Dataset" # The dataset name on Hugging Face Hub
model_path = Path().resolve().parent.parent / "saved_models" / f"{model_id.split('/')[-1]}_instruct" # Path to save the trained model to

In [ ]:
# Set the seed for reproducibility
set_seed(seed)

### Data loading

In [ ]:
# Load the dataset from Hugging Face Hub
dataset = load_hf_dataset(dataset_name, split="train")

In [ ]:
# Convert the dataset to a pandas DataFrame for easier manipulation
dataset_df = dataset_to_pandas(dataset)

# Display the first few rows of the dataset
display(dataset_df.head())

### Preprocess data

In [ ]:
# Train-test split
train_dataset, test_dataset = dataset.train_test_split(test_size=test_size, seed=seed).values()

### Tokenizer

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set the padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
# Define the chat template if not already defined
if not tokenizer.chat_template:
	tokenizer.chat_template = """{% for message in messages %}
	{% if message['role'] == 'system' %}System: {{ message['content'] }}\n
	{% elif message['role'] == 'user' %}User: {{ message['content'] }}\n
	{% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
	{% endif %}
	{% endfor %}"""

### Building the model

In [ ]:
# Define the quantization configurations of the model (only for CUDA devices)
quantization_config = None
if use_cuda:
    quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        bnb_4bit_use_double_quant = True
    )

In [ ]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage = True,
    quantization_config = quantization_config,
    device_map = "auto"
)

### Training the model

In [ ]:
# Mixed precision settings
use_pin_memory = bool(use_cuda)
bf16 = bool(use_cuda and torch.cuda.is_bf16_supported())

# SFTTrainer config 
sft_config = SFTConfig(
    learning_rate = 5e-5,
    num_train_epochs = 5,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps = 16,
    logging_steps = 10,
    eval_strategy = "steps",
    dataloader_pin_memory = use_pin_memory,
	bf16 = bf16,
	weight_decay = 0.01,
 	lr_scheduler_type = "linear"
)

In [ ]:
# Initialize the SFTTrainer
sft_trainer = SFTTrainer(
    model = model,
    args = sft_config,
    train_dataset = train_dataset, 
    eval_dataset = test_dataset,
    processing_class = tokenizer
)

# Start the training process
sft_trainer.train()

### Inference

In [ ]:
def generate_responses(model, tokenizer, user_message: str, system_message: Optional[str] = None, max_new_tokens: int = 100) -> str:
    # Format chat using tokenizer's chat template
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    
    # Add user message
    messages.append({"role": "user", "content": user_message})
        
    # Generate the prompt using the chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True,
        enable_thinking = False
    )

    # Tokenize the prompt and move to the appropriate device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Disable gradient calculations for inference
    with torch.no_grad():
        # Generate the model outputs
        outputs = model.generate(
            **inputs,
            max_new_tokens = max_new_tokens,
            do_sample = False,
            pad_token_id = tokenizer.eos_token_id,
            eos_token_id = tokenizer.eos_token_id,
        )
        
    # Decode the generated tokens to get the response
    input_len = inputs["input_ids"].shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    # Return the generated response
    return response


def test_model_with_questions(model, tokenizer, questions: list[str], system_message: Optional[str] = None, title: str = "Model Output"):
    # Test the model with a list of questions and print the responses
    print(f"\n=== {title} ===")
    
    # Iterate through each question and generate a response
    for i, question in enumerate(questions, 1):
        # Generate the response
        response = generate_responses(model, tokenizer, question, system_message)
        
        # Print the input question and the model's response
        print(f"\nModel Input {i}:\n{question}\nModel Output {i}:\n{response}\n")

In [ ]:
# Define a list of questions to test the model
questions = [
    "Give me an 1-sentence introduction of LLM.",
    "Calculate 1+1-1",
    "What's the difference between thread and process?"
]

# Test the fine-tuned model with the defined questions
test_model_with_questions(sft_trainer.model, tokenizer, questions, title="Base Model (After SFT) Output")